# Tutorial: Uso de AWS Lambda con Amazon Kinesis

En este tutorial, creará una función de Lambda para consumir eventos de un flujo de datos de Kinesis.

* Una aplicación personalizada escribe los registros en el flujo.

* AWS Lambda sondea el flujo y, cuando detecta registros nuevos en él, llama a la función de Lambda.

* AWS Lambda ejecuta la función de Lambda asumiendo el rol de ejecución que se especificó en el momento de crear la función de Lambda.

In [1]:
import boto3
import json
from zipfile import ZipFile

lambda_client = boto3.client('lambda')
iam = boto3.client('iam')
kinesis_client = boto3.client('kinesis')
account_id = boto3.client('sts').get_caller_identity().get('Account')


Cree el rol de ejecución que concederá a su función permiso para obtener acceso a los recursos de AWS.

In [3]:
# Create the role
role_name = 'LambdaToKinesisRole'

role = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps({
        'Version': '2012-10-17',
        'Statement': [{
            'Effect': 'Allow',
            'Principal': {
                'Service': 'lambda.amazonaws.com'
            },
            'Action': 'sts:AssumeRole'
        }]
    })
)

# Attach the policy to the role
response = iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaKinesisExecutionRole'
)

if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('Role created successfully')

Role created successfully


In [4]:
%%writefile lambda_function.py
import json
import base64

print('Loading function')

def lambda_handler(event, context):
    #print(json.dumps(event, indent=2))
    for record in event['Records']:
        # Kinesis data is base64 encoded so decode here
        decode_record = base64.b64decode(record['kinesis']['data'])
        payload = decode_record.decode('utf-8')
        print('Decoded payload:', payload)

Overwriting lambda_function.py


In [5]:
# Create the zip file
with ZipFile('lambda_function.zip', 'w') as myzip:
    myzip.write('lambda_function.py')
# Read the zip file into memory
with open('lambda_function.zip', 'rb') as f:
    zipped_code = f.read()

In [6]:
# Create the Lambda function
response = lambda_client.create_function(
    FunctionName='KinesisLambdaFunction',
    Runtime='python3.9',
    Role='arn:aws:iam::{}:role/{}'.format(account_id, role_name),
    Handler='lambda_function.lambda_handler',
    Code={
        'ZipFile': zipped_code
    },
    Description='Lambda function to process Kinesis data',
    Timeout=3,
    MemorySize=128,
    Publish=True,
    Environment={
        'Variables': {
            'ENVIRONMENT': 'DEV'
        }
    }
)

In [7]:
# Read the test event
with open('test_event.json', 'r') as f:
    test_event = json.load(f)

In [9]:
# Test the Lambda function
response = lambda_client.invoke(
    FunctionName='KinesisLambda',
    InvocationType='RequestResponse',
    Payload=json.dumps(test_event)
)

if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('Lambda function invoked successfully')

Lambda function invoked successfully


Utilice la función `create_stream()` para crear un flujo.

In [10]:
# Create the Kinesis stream
response = kinesis_client.create_stream(
    StreamName='KinesisStreamToLambda',
    ShardCount=1
)

In [11]:
# Get the ARN of the Kiensis stream
kinesis_arn = kinesis_client.describe_stream(StreamName='KinesisStreamToLambda')['StreamDescription']['StreamARN']
print(kinesis_arn)

arn:aws:kinesis:us-east-1:448244794999:stream/KinesisStreamToLambda


In [12]:
# Add event to the Lambda function
response = lambda_client.create_event_source_mapping(
    FunctionName='KinesisLambdaFunction',
    EventSourceArn=kinesis_arn,
    BatchSize=100,
    StartingPosition='LATEST'
)

In [13]:
# List the event sources mapped to the Lambda function
kinesis_mapped = lambda_client.list_event_source_mappings(
    FunctionName='KinesisLambdaFunction'
)
print("Status: " + kinesis_mapped['EventSourceMappings'][0]['State'])

Status: Enabled


En la respuesta, puede verificar que el valor de estado es enabled. Los mapeos de orígenes de eventos se pueden deshabilitar para poner en pausa temporalmente el sondeo sin perder de registros.

Para probar el mapeo de origen de eventos, agregue los registros de eventos a su flujo de Kinesis.

In [31]:
# Put records into the Kinesis stream	
response = kinesis_client.put_records(
    Records=[
        {
            'Data': 'Hello, this is a test number 1',
            'PartitionKey': '1'
        },
        {
            'Data': 'Hello, this is a test number 2',
            'PartitionKey': '2'
        },
        {
            'Data': 'Hello, this is a test number 3',
            'PartitionKey': '3'
        }
    ],
    StreamName='KinesisStream'
)

In [20]:
# List all cloudwatch logs
logs_client = boto3.client('logs')
response = logs_client.describe_log_groups()
for log in response['logGroups']:
    # If log contain kinesis, print the log stream
    if 'Kinesis' in log['logGroupName']:
        log_group_name = log['logGroupName']

Lambda utiliza el rol de ejecución para leer los registros desde el flujo. A continuación, se invoca la función de Lambda y se pasan lotes de registros. La función descodifica los datos de cada registro y los registra, enviando la salida a CloudWatch Logs. Puede ver los registros en la consola de CloudWatch.

In [40]:
# Print cloudwatch logs from the Lambda function
logs_client = boto3.client('logs')
log_stream_name = logs_client.describe_log_streams(
    logGroupName=log_group_name,
    orderBy='LastEventTime',
    descending=True
)['logStreams'][0]['logStreamName']

response = logs_client.get_log_events(
    logGroupName=log_group_name,
    logStreamName=log_stream_name
)

for event in response['events']:
    print(event['message'])

START RequestId: a4a0264f-4715-4d79-a56d-9dd4027d98b3 Version: $LATEST

Decoded payload: Hello, this is a test number 1

Decoded payload: Hello, this is a test number 2

Decoded payload: Hello, this is a test number 3

END RequestId: a4a0264f-4715-4d79-a56d-9dd4027d98b3

REPORT RequestId: a4a0264f-4715-4d79-a56d-9dd4027d98b3	Duration: 11.55 ms	Billed Duration: 12 ms	Memory Size: 128 MB	Max Memory Used: 37 MB	

